In [32]:
import pandas as pd

In [38]:
df = pd.read_csv("db_ingestion/nasdaq_data/nasdaq_screener_1638944075113.csv")

In [41]:
df = df[df['Sector'] == 'Capital Goods']
stock_list = list(df['Symbol'])

In [42]:
stock_list

['A',
 'AAON',
 'AATC',
 'ABNB',
 'ACA',
 'ACHR',
 'ADNT',
 'ADTN',
 'AEHL',
 'AEHR',
 'AENZ',
 'AERC',
 'AEVA',
 'AEY',
 'AFI',
 'AGCO',
 'AGX',
 'AIMC',
 'AIR',
 'AIRI',
 'AIT',
 'AJRD',
 'AKYA',
 'ALG',
 'ALPP',
 'ALSN',
 'ALV',
 'AMOT',
 'AMPG',
 'AMPGW',
 'AMSC',
 'AOS',
 'AP',
 'APH',
 'APTV',
 'AQUA',
 'ARTW',
 'ARVL',
 'ASPN',
 'ASTC',
 'ASTE',
 'ATER',
 'ATR',
 'ATRO',
 'AVAV',
 'AVNW',
 'AXDX',
 'AXL',
 'AZZ',
 'B',
 'BA',
 'BBU',
 'BECN',
 'BELFA',
 'BELFB',
 'BGRY',
 'BGRYW',
 'BHTG',
 'BIO',
 'BKSY',
 'BLBD',
 'BLD',
 'BMI',
 'BNGO',
 'BNGOW',
 'BNSO',
 'BODY',
 'BOXL',
 'BRKR',
 'BWA',
 'BWEN',
 'BWSN',
 'BXC',
 'BZH',
 'CAAS',
 'CAMT',
 'CARR',
 'CAT',
 'CBAT',
 'CCAC',
 'CCS',
 'CECE',
 'CFX',
 'CFXA',
 'CGNX',
 'CHX',
 'CIR',
 'CMCO',
 'CMI',
 'CNHI',
 'CNM',
 'CNR',
 'CODA',
 'COHR',
 'COHU',
 'COOK',
 'CPAC',
 'CPNG',
 'CPS',
 'CR',
 'CRCT',
 'CRH',
 'CSTE',
 'CTS',
 'CVGI',
 'CVU',
 'CW',
 'CX',
 'CYBE',
 'CYD',
 'DAIO',
 'DAKT',
 'DAN',
 'DASH',
 'DCI',
 'DCO',
 'D

In [36]:
options = []
for i in lst:
    df_filtered = df[df['Sector'] == i]
    lst = list(df_filtered['Symbol'])
    options.append({'label':i, 'value':lst})

In [37]:
print(options)

[{'label': 'Capital Goods', 'value': ['A', 'AAON', 'AATC', 'ABNB', 'ACA', 'ACHR', 'ADNT', 'ADTN', 'AEHL', 'AEHR', 'AENZ', 'AERC', 'AEVA', 'AEY', 'AFI', 'AGCO', 'AGX', 'AIMC', 'AIR', 'AIRI', 'AIT', 'AJRD', 'AKYA', 'ALG', 'ALPP', 'ALSN', 'ALV', 'AMOT', 'AMPG', 'AMPGW', 'AMSC', 'AOS', 'AP', 'APH', 'APTV', 'AQUA', 'ARTW', 'ARVL', 'ASPN', 'ASTC', 'ASTE', 'ATER', 'ATR', 'ATRO', 'AVAV', 'AVNW', 'AXDX', 'AXL', 'AZZ', 'B', 'BA', 'BBU', 'BECN', 'BELFA', 'BELFB', 'BGRY', 'BGRYW', 'BHTG', 'BIO', 'BKSY', 'BLBD', 'BLD', 'BMI', 'BNGO', 'BNGOW', 'BNSO', 'BODY', 'BOXL', 'BRKR', 'BWA', 'BWEN', 'BWSN', 'BXC', 'BZH', 'CAAS', 'CAMT', 'CARR', 'CAT', 'CBAT', 'CCAC', 'CCS', 'CECE', 'CFX', 'CFXA', 'CGNX', 'CHX', 'CIR', 'CMCO', 'CMI', 'CNHI', 'CNM', 'CNR', 'CODA', 'COHR', 'COHU', 'COOK', 'CPAC', 'CPNG', 'CPS', 'CR', 'CRCT', 'CRH', 'CSTE', 'CTS', 'CVGI', 'CVU', 'CW', 'CX', 'CYBE', 'CYD', 'DAIO', 'DAKT', 'DAN', 'DASH', 'DCI', 'DCO', 'DD', 'DE', 'DORM', 'DOV', 'DPW', 'DXPE', 'ELMS', 'ELMSW', 'ELSE', 'EMAN', 'EME',